# Discussion Section Week 3

## Code Vectorization

### Inner Product Example

In [2]:
import numpy as np
n = 50000000
A = np.random.rand(n)
B = np.random.rand(n)

Computing the inner product with a loop vs using numpy. Can you guess the order of magnitude difference between the two?

In [ ]:
%%time
t1 = 0
for i in range(n):
    t1 += A[i] * B[i]

In [ ]:
%%time
t2 = A @ B

In [ ]:
print("Value of t1 is %d and t2 is %d" % (t1,t2))

### Numpy Indexing

In [39]:
A = np.arange(9).reshape(3,3)
print(A)

[[0 1 2]
 [3 4 5]
 [6 7 8]]


How do I get:

a) the first column of A (column of index zero)?

b) the last row of A?

c) the element at index (0,0) and (2,2) concatenated into an array?

d) the bottom two elements in the last column?

array([5, 8])

"Fun" questions:

a) How do I get just the corner values into a matrix? Output should be: [[0, 2],[6, 8]]

b) The last two elements of the first and last row? Output: [[1,2],[7,8]]

In [35]:
A[0::2,0::2]

array([[0, 2],
       [6, 8]])

In [36]:
A[[0,2],1:]

array([[1, 2],
       [7, 8]])

### Matrix Permutations

Matrix row permutations with a loop vs permutation vector.

In [7]:
m = 10000
M = np.round(np.random.rand(m,m) * 10, 0)
A = M
B = M

In [8]:
%%time
# Permute rows 1 and 2
for i in range(m):
    tempNumber = A[2,i]
    A[2,i] = A[1,i]
    A[1,i] = tempNumber

CPU times: user 8.43 ms, sys: 575 µs, total: 9 ms
Wall time: 8.81 ms


In [9]:
%%time
B = M
p = np.arange(m)
p[1] = 2
p[2] = 1
B = B[p,:]

CPU times: user 361 ms, sys: 376 ms, total: 737 ms
Wall time: 750 ms


In [10]:
np.allclose(A,B)

False

Why are they not the same??

In [11]:
import copy
m = 10000
M = np.round(np.random.rand(m,m) * 10, 0)
A = copy.deepcopy(M)
B = copy.deepcopy(M)

In [12]:
%%time
# Permute rows 1 and 2
for i in range(m):
    tempNumber = A[2,i]
    A[2,i] = A[1,i]
    A[1,i] = tempNumber

CPU times: user 7.95 ms, sys: 1.33 ms, total: 9.27 ms
Wall time: 8.16 ms


In [13]:
%%time
B = M
p = np.arange(m)
p[1] = 2
p[2] = 1
B = B[p,:]

CPU times: user 294 ms, sys: 368 ms, total: 662 ms
Wall time: 664 ms


In [14]:
np.allclose(A,B)

True

The permutation vector is slower?

In [15]:
import copy
m = 10000
M = np.round(np.random.rand(m,m) * 10, 0)

In [17]:
%%time
A = copy.deepcopy(M)
# Permute rows 1 and 2
for i in range(m):
    tempNumber = A[2,i]
    A[2,i] = A[1,i]
    A[1,i] = tempNumber

CPU times: user 346 ms, sys: 350 ms, total: 696 ms
Wall time: 702 ms


In [20]:
%%time
B = M
p = np.arange(m)
p[1] = 2
p[2] = 1
B = B[p,:]

CPU times: user 403 ms, sys: 961 ms, total: 1.36 s
Wall time: 1.37 s


In [21]:
np.allclose(A,B)

True

It turns out that the permutation vector method involves a deepcopy of B.

Moral of the story? Vectorization does not always help, especially when it comes to memory operations, but for arithmatic it general does.

### LU factorization pivoting

In [67]:
A = np.array([[1,8,1],[2,2,1],[1,10,3]])

In [72]:
def LUfactor(A, pivoting = True):
    """Factor a square matrix with partial pivoting, A[p,:] == L @ U
    Parameters: 
      A: the matrix.
      pivoting = True: whether or not to do partial pivoting
    Outputs (in order):
      L: the lower triangular factor, same dimensions as A, with ones on the diagonal
      U: the upper triangular factor, same dimensions as A
      p: the permutation vector that permutes the rows of A by partial pivoting
    """
    # Check the input
    m, n = A.shape
    assert m == n, 'input matrix A must be square'
    
    # Initialize p to be the identity permutation
    p = np.array(range(n))
    
    # Make a copy of the matrix that we will transform into L and U
    LU = A.astype(np.float64).copy()
    
    # Eliminate each column in turn
    for piv_col in range(n):
        
        # Choose the pivot row and swap it into place
        if pivoting:
            piv_row = piv_col + np.argmax(np.abs(LU[piv_col:, piv_col]))
            assert LU[piv_row, piv_col] != 0., "can't find nonzero pivot, matrix is singular"
            LU[[piv_col, piv_row], :]  = LU[[piv_row, piv_col], :]
            p[[piv_col, piv_row]]      = p[[piv_row, piv_col]]
            
        # Update the rest of the matrix
        pivot = LU[piv_col, piv_col]
        assert pivot != 0., "pivot is zero, can't continue"
        for row in range(piv_col + 1, n):
            multiplier = LU[row, piv_col] / pivot
            LU[row, piv_col] = multiplier
            LU[row, (piv_col+1):] -= multiplier * LU[piv_col, (piv_col+1):]
            
    # Separate L and U in the result
    U = np.triu(LU)
    L = LU - U + np.eye(n)
    
    return (L, U, p)

In [73]:
L, U, p = LUfactor(A)

In [74]:
p

array([1, 2, 0])

In [75]:
L

array([[1.        , 0.        , 0.        ],
       [0.5       , 1.        , 0.        ],
       [0.5       , 0.77777778, 1.        ]])

In [76]:
U

array([[ 2.        ,  2.        ,  1.        ],
       [ 0.        ,  9.        ,  2.5       ],
       [ 0.        ,  0.        , -1.44444444]])

In [77]:
T = L@U

In [80]:
T

array([[ 2.,  2.,  1.],
       [ 1., 10.,  3.],
       [ 1.,  8.,  1.]])

In [81]:
A[p,:]

array([[ 2,  2,  1],
       [ 1, 10,  3],
       [ 1,  8,  1]])